In [5]:
##############################################################
#Part I of this big program
#This program for subtask 2 on laptop only
#We only read Train_laptop.csv 
#we will get 4 csv files containing sentiment adj 
#then delete some error adj by hand and carry on next program
##############################################################

import csv
from time import time

                    #############################################
                    ############  I. Read CSV files  ############
                    #############################################
start=time()
###########  read Train_laptop.csv ###########
#I-1: get lists
trainSent=[]
trainAT=[]
trainSentsNum=0
with open('/Users/xizhaohan/Desktop/XML_to_CSV/Train_laptop.csv', 'r') as file_trainLPT:
    reader_trainLPT = csv.reader(file_trainLPT)
    for row in reader_trainLPT:
    # row is a list:
    #['(AT1#ATP1)(AT2#ATP2)', 'sent']
        trainSent.append(row[1])
        ATraw=row[0].strip('()').split(')(')
        samesentAT=[tuple(ele1.split('#')) for ele1 in ATraw]
        trainAT.append(samesentAT)
        #trainAT are 3-D structure:
        #most inside-D is ('AT','ATP') pair
        #middle D is pair from same sentence: [(pair),(pair)]
        #outside looks like:[[same sent],[same sent]]
        trainSentsNum+=1
file_trainLPT.close()

#I-2: get total dictionary
TrainAll_dict={}
for ID in range(1,trainSentsNum+1):
    #ID in dict form 1 to 3041(total number of training sentences)
    TrainAll_dict[str(ID)]={'sentence':trainSent[ID-1], 'AspectTerm':trainAT[ID-1]}
    #TrainAll_dict is a 2-D dictionary:
    #{'ID1':{'sentence':'sent','AspectTerm':[(AT1,ATP1),(AT2,ATP2)]}, 'ID2':{}, ...}
                  
                #######################################################
                ############  II. Get 4 sentiment classes  ############
                #######################################################

from nltk.parse.stanford import StanfordDependencyParser
path_to_jar = '/Users/xizhaohan/Desktop/Text Mining/project1/My work/stanford-parser-full-2014-08-27/stanford-parser.jar'
path_to_models_jar = '/Users/xizhaohan/Desktop/Text Mining/project1/My work/stanford-parser-full-2014-08-27/stanford-parser-3.4.1-models.jar'
dependency_parser = StanfordDependencyParser(path_to_jar=path_to_jar, path_to_models_jar=path_to_models_jar)

SentimentDict={'positive':[],'negative':[], 'neutral':[] , 'conflict':[]}

for ID in range(1,trainSentsNum+1):
    #if ID>5:
        #break
    #II-1: drag element from trainAll_dict
    currentSent=TrainAll_dict[str(ID)]['sentence']
    currentAT_and_P=TrainAll_dict[str(ID)]['AspectTerm']
    if len(currentAT_and_P[0])>1:
        currentAT=[p[0] for p in currentAT_and_P] #store all AT in current sentence, 1-D list
        currentATP=[p[1] for p in currentAT_and_P] #store all ATP in current sentence, 1-D list
        currentAT_P_dict={} #store all AT and ATP in {'AT': ATP} form, 1-D dictionary
        for at_p_pair in currentAT_and_P:
            currentAT_P_dict[at_p_pair[0]]=at_p_pair[1]
    else:
        currentAT=[]
        currentATP=[]
        currentAT_P_dict={}
        
    #II-2 parse each sentence and get all (N,Adj) pairs in list form
    Ntag=('NN','NNS','NNP','NNPS')
    ADJtag=('JJ','JJR','JJS')
    N_Adj_pair=[]
    try:
        ParseSent=list(dependency_parser.raw_parse(currentSent).__next__().triples())
    except BaseException:  continue
    #ParseSent looks like:
    #[(('horrible', 'JJ'), 'cc', ('But', 'CC')), 
    #(('horrible', 'JJ'), 'nsubj', ('staff', 'NN')), 
    #(('staff', 'NN'), 'det', ('the', 'DT')), 
    #(('to', 'TO'), 'pobj', ('us', 'PRP'))]
    for outpair in ParseSent:
        if outpair[0][1] in Ntag and outpair[2][1] in ADJtag:
            N_Adj_pair.append(tuple((outpair[0][0], outpair[2][0])))
        elif outpair[2][1] in Ntag and outpair[0][1] in ADJtag:
            N_Adj_pair.append(tuple((outpair[2][0], outpair[0][0])))
    #N_Adj_pair looks like:
    #[('food', 'fair'), ('factor', 'only'), ('deficiencies', 'other')]
    
    #II-3: detect adj
    if len(set(currentATP))==1: #len of set(currentATP) is kinds of polarity of AT, is 0,1 or 2
        SentimentDict[currentATP[0]]+=[ele[1] for ele in N_Adj_pair]
    elif len(set(currentATP))>1:
        for pair in N_Adj_pair:
            noun=pair[0]
            adj=pair[1]
            for at in currentAT:
                if noun==at or noun in at:
                    SentimentDict[currentAT_P_dict[at]].append(adj)
for key in SentimentDict:
    SentimentDict[key]=list(set(SentimentDict[key]))

#II-4: write into csv file and delete by hand
#only delete some adj obviously not in this sentiment
#then use for enxtend
with open( '/Users/xizhaohan/Desktop/Subtask2_4/positive_not_enxtend_from_trainLPT.csv', 'w') as file:
    writer = csv.writer(file)
    for col1 in SentimentDict['positive']:
        writer.writerow([str(col1)])
file.close()

with open( '/Users/xizhaohan/Desktop/Subtask2_4/negative_not_enxtend_from_trainLPT.csv', 'w') as file:
    writer = csv.writer(file)
    for col1 in SentimentDict['negative']:
        writer.writerow([str(col1)])
file.close()

with open( '/Users/xizhaohan/Desktop/Subtask2_4/neutral_not_enxtend_from_trainLPT.csv', 'w') as file:
    writer = csv.writer(file)
    for col1 in SentimentDict['neutral']:
        writer.writerow([str(col1)])
file.close()

with open( '/Users/xizhaohan/Desktop/Subtask2_4/conflict_not_enxtend_from_trainLPT.csv', 'w') as file:
    writer = csv.writer(file)
    for col1 in SentimentDict['conflict']:
        writer.writerow([str(col1)])
file.close()

stop=time()
print('Total time is:',stop-start,'seconds')
#this part of program end

//anaconda/lib/python3.5/site-packages/nltk/parse/dependencygraph.py:378: UserWarning: The graph doesn't contain a node that depends on the root element.
  "The graph doesn't contain a node "


Total time is: 2963.4731209278107 seconds


In [2]:
##############################################################
#Part II of this big program
#This program for subtask 2 on laptop only
#We read Test_laptop_PhaseB.csv and 4 deleted CSV files
##############################################################

import csv
from time import time
from textblob import TextBlob
from nltk.corpus import wordnet as wn
from nltk.parse.stanford import StanfordDependencyParser
path_to_jar = '/Users/xizhaohan/Desktop/Text Mining/project1/My work/stanford-parser-full-2014-08-27/stanford-parser.jar'
path_to_models_jar = '/Users/xizhaohan/Desktop/Text Mining/project1/My work/stanford-parser-full-2014-08-27/stanford-parser-3.4.1-models.jar'
dependency_parser = StanfordDependencyParser(path_to_jar=path_to_jar, path_to_models_jar=path_to_models_jar)

                    #############################################
                    ############  I. Read CSV files  ############
                    #############################################
start=time()
###########  I-1:read Test_laptop.csv ###########
#I-1-1: get lists
testSent=[]
testAT=[]
testSentsNum=0
with open('/Users/xizhaohan/Desktop/XML_to_CSV/Test_laptop_PhaseB.csv', 'r') as file_testLPT:
    reader_testLPT = csv.reader(file_testLPT)
    for row in reader_testLPT:
    # row is a list:
    #['(AT1)(AT2)', 'sent']
        testSent.append(row[1])
        ATraw=row[0].strip('()').split(')(')
        samesentAT=[ele1 for ele1 in ATraw]
        testAT.append(samesentAT)
        #testAT are 2-D structure:
        #inside D is AT from same sentence: [AT1,AT2,...]
        #outside looks like:[[same sent],[same sent]]
        testSentsNum+=1
file_testLPT.close()

#I-1-2: get total dictionary
TestAll_dict={}
for ID in range(1,testSentsNum+1):
    #ID in dict form 1 to 800(total number of testing sentences)
    TestAll_dict[str(ID)]={'sentence':testSent[ID-1], 'AspectTerm':testAT[ID-1]}
    #TestAll_dict is a 2-D dictionary:
    #{'ID1':{'sentence':'sent','AspectTerm':[AT1,AT2]}, 'ID2':{}, ...}

###########  I-2:read 4 CSV files with deleted sentiment adj ###########
SentimentDict_Deleted={'positive':[],'negative':[], 'neutral':[] , 'conflict':[]}
with open('/Users/xizhaohan/Desktop/Subtask2_4/positive_deleted_LPT.csv', 'r') as file1:
    reader1 = csv.reader(file1)
    for row in reader1:
        SentimentDict_Deleted['positive']+=row
file1.close()
with open('/Users/xizhaohan/Desktop/Subtask2_4/negative_deleted_LPT.csv', 'r') as file2:
    reader2 = csv.reader(file2)
    for row in reader2:
        SentimentDict_Deleted['negative']+=row
file2.close()
with open('/Users/xizhaohan/Desktop/Subtask2_4/neutral_deleted_LPT.csv', 'r') as file3:
    reader3 = csv.reader(file3)
    for row in reader3:
        SentimentDict_Deleted['neutral']+=row
file3.close()
with open('/Users/xizhaohan/Desktop/Subtask2_4/conflict_deleted_LPT.csv', 'r') as file4:
    reader4 = csv.reader(file4)
    for row in reader4:
        SentimentDict_Deleted['conflict']+=row
file4.close()


                    ########################################################
                    ############  II. Extend 4 sentiment lists  ############
                    ########################################################

SentimentDict_Extended={'positive':[],'negative':[], 'neutral':[] , 'conflict':[]}
for adj in SentimentDict_Deleted['positive']:
    allsyn=wn.synsets(adj)
    for word in allsyn:
        SentimentDict_Extended['positive'].append(str(word).strip('Synset()').split('.')[0].strip('\''))
SentimentDict_Extended['positive']=list(set(SentimentDict_Extended['positive']))

for adj in SentimentDict_Deleted['negative']:
    allsyn=wn.synsets(adj)
    for word in allsyn:
        SentimentDict_Extended['negative'].append(str(word).strip('Synset()').split('.')[0].strip('\''))
SentimentDict_Extended['negative']=list(set(SentimentDict_Extended['negative']))

SentimentDict_Extended['neutral']=SentimentDict_Deleted['neutral']
SentimentDict_Extended['conflict']=SentimentDict_Deleted['conflict']

                    ##########################################################
                    ############  III. Judge polarity of test AT  ############
                    ##########################################################

OutTestLPTDict={}
for ID in range(1,testSentsNum+1):
    #if ID>30:
        #break
    #III-1: drag element from testAll_dict
    currentSent=TestAll_dict[str(ID)]['sentence']
    currentAT=TestAll_dict[str(ID)]['AspectTerm']
    OutTestLPTDict[str(ID)]={'sentence':currentSent, 'AspectTerm':[]}
    if len(currentAT[0])==0:
        continue
    #III-2 parse each sentence and get all (N,Adj) pairs in list form
    Ntag=('NN','NNS','NNP','NNPS')
    ADJtag=('JJ','JJR','JJS')
    N_Adj_pair=[]
    try:
        ParseSent=list(dependency_parser.raw_parse(currentSent).__next__().triples())
    except BaseException:  continue
    #ParseSent looks like:
    #[(('horrible', 'JJ'), 'cc', ('But', 'CC')), 
    #(('horrible', 'JJ'), 'nsubj', ('staff', 'NN')), 
    #(('staff', 'NN'), 'det', ('the', 'DT')), 
    #(('to', 'TO'), 'pobj', ('us', 'PRP'))]
    for outpair in ParseSent:
        if outpair[0][1] in Ntag and outpair[2][1] in ADJtag:
            N_Adj_pair.append(tuple((outpair[0][0], outpair[2][0])))
        elif outpair[2][1] in Ntag and outpair[0][1] in ADJtag:
            N_Adj_pair.append(tuple((outpair[2][0], outpair[0][0])))
    #N_Adj_pair looks like:
    #[('food', 'fair'), ('factor', 'only'), ('deficiencies', 'other')]
    
    #III-3: detect whether N is AT
    AT_ADJ=[]
    for pair in N_Adj_pair:
        noun=pair[0]
        adj=pair[1]
        for at in currentAT:
            if noun==at or noun in at:
                AT_ADJ.append(tuple((at, adj)))
    #AT_ADJ looks like:
    #[('sushi', 'best'), ('place', 'clean')]
    #its noun is real AT, its adj is candidate
    #III-4: detect polarity of AT in AT_ADJ by adj
    AspectTerm=[]
    for p in AT_ADJ:
        if p[1] in SentimentDict_Extended['positive']:
            AspectTerm.append(tuple((p[0], 'positive')))
        elif p[1] in SentimentDict_Extended['negative']:
            AspectTerm.append(tuple((p[0], 'negative')))
        elif p[1] in SentimentDict_Extended['conflict']:
            AspectTerm.append(tuple((p[0], 'conflict')))
        else:
            AspectTerm.append(tuple((p[0], 'neutral')))
    if len(AT_ADJ)!=0:
        OutTestLPTDict[str(ID)]['AspectTerm']=AspectTerm
        #OutTestRSTDict store the outcome of this program in dictionary form,looks like:
        #{'id1':{'sentence':..., 'AspectTerm':[(AT1,ATP1),(AT2,ATP2)], 'AspectCategory':[AC1,AC2]}, 'id2':{}}
    else:
        blob = TextBlob(currentSent)
        tempATP=blob.sentiment.polarity
        if tempATP>0:
            for term in currentAT:
                AspectTerm.append(tuple((term, 'positive')))
        elif tempATP<0:
            for term in currentAT:
                AspectTerm.append(tuple((term, 'negative')))
        else:
            for term in currentAT:
                AspectTerm.append(tuple((term, 'neutral')))
        OutTestLPTDict[str(ID)]['AspectTerm']=AspectTerm   
    OutTestLPTDict[str(ID)]['AspectTerm']=list(set(OutTestLPTDict[str(ID)]['AspectTerm']))
    
                    ##############################################
                    ############  IV. Write into CSV  ############
                    ##############################################               
AT_Write=[]
Sent_Write=[]
for ID in range(1,testSentsNum+1):
    #if ID>30:
        #break
    Sent_Write.append(OutTestLPTDict[str(ID)]['sentence'])
    sameSentAT=''
    for ele1 in OutTestLPTDict[str(ID)]['AspectTerm']:
        sameSentAT+='('+ele1[0]+'#'+ele1[1]+')'
    AT_Write.append(sameSentAT)

with open( '/Users/xizhaohan/Desktop/Subtask2_4/Subtask2_FinalOutcome_laptop.csv', 'w') as file:
    writer = csv.writer(file)
    for col1,col2 in zip(AT_Write,Sent_Write):
        writer.writerow([str(col1),str(col2)])
file.close()

stop=time()
print('Total time is:', stop-start,'seconds')

#Subtask2 for laptop test PhaseB is finished
#we got an CSV file with polarities of AT
#this is the final outcome for laptop


Total time is: 404.71827507019043 seconds
